# Day 25
https://adventofcode.com/2016/day/25

In [1]:
import aocd
data = aocd.get_data(year=2016, day=25)

In [2]:
from collections import deque

In [4]:
def get(registers, source):
    try:
        return int(source)
    except:
        return registers.get(source)

In [5]:
def cpy(registers, source, dest):
    if dest in registers:
        registers[dest] = get(registers, source)

def inc(registers, register):
    if register in registers:
        registers[register] += 1

def dec(registers, register):
    if register in registers:
        registers[register] -= 1

def add(registers, source, dest):
    if dest in registers:
        registers[dest] = get(registers, dest) + get(registers, source)

def mul(registers, source, dest):
    if dest in registers:
        registers[dest] = get(registers, dest) * get(registers, source)

def tgl(instruction):
    cmd, *args = instruction

    if len(args) == 1:
        if cmd == 'inc':
            return ['dec', *args]
        else:
            return ['inc', *args]
    elif len(args) == 2:
        if cmd == 'jnz':
            return ['cpy', *args]
        else:
            return ['jnz', *args]

In [6]:
def patched_instructions(instructions, current):
    consider = instructions[current:current+3]
    if (len(consider) == 3 and
        consider[0][0] == 'dec' and
        consider[1][0] == 'inc' and
        consider[0][1] != consider[1][1] and
        consider[2][0] == 'jnz' and
        consider[2][1] == consider[0][1] and
        consider[2][2] == '-2'):
        return [
            ['add', consider[0][1], consider[1][1]],
            ['cpy', '0', consider[0][1]],
            ['jnz', '0', '0']
        ]

    consider = instructions[current:current+6]
    if (len(consider) == 6 and
        consider[0][0] == 'cpy' and
        consider[1][0] == 'inc' and
        consider[2][0] == 'dec' and
        consider[3][0] == 'jnz' and
        consider[3][2] == '-2' and
        consider[4][0] == 'dec' and
        consider[5][0] == 'jnz' and
        consider[5][2] == '-5'):
        return [
            ['mul', consider[0][1], consider[4][1]],
            ['add', consider[4][1], consider[1][1]],
            ['cpy', '0', consider[0][2]],
            ['cpy', '0', consider[4][1]],
            ['jnz', '0', '0'],
            ['jnz', '0', '0']
        ]

    return []

In [7]:
def run_program(text, **kwargs):
    registers = dict(a=0, b=0, c=0, d=0)
    registers.update(kwargs)
    instructions = [instruct.split() for instruct in text.split('\n')]
    current = 0
    do = dict(cpy=cpy, inc=inc, dec=dec, add=add, mul=mul)
    replaced_instructions = deque()

    while current < len(instructions):
        replaced_instructions.extend(patched_instructions(instructions, current))
        if replaced_instructions:
            cmd, *args = replaced_instructions.popleft()
        else:
            cmd, *args = instructions[current]

        if cmd in do:
            do[cmd](registers, *args)
        elif cmd == 'jnz':
            if get(registers, args[0]) != 0:
                current += get(registers, args[1]) - 1
        elif cmd == 'tgl':
            target = current + get(registers, args[0])
            if target >=0 and target < len(instructions):
                instructions[target] = tgl(instructions[target])
        elif cmd == 'out':
            yield get(registers, args[0])

        current += 1

In [10]:
def find_starting_a_for_clock_output(text):
    test = 0

    while True:
        test += 1
        output = run_program(text, a=test)

        for i in range(10):
            nxt = next(output)
            if nxt != (i % 2):
                break
            if i == 9:
                return test

In [11]:
p1 = find_starting_a_for_clock_output(data)
print('Part 1: {}'.format(p1))

Part 1: 196
